# Backtesting the HMM Model

Signals are identifed by the hmm model and imported into the system. 

    When signal = 1, we enter the mean reversion position.    
    When signal = 0, we enter the momentum position. 


Mean Reversion Strategy:

     1.If return in the past 21 days > 0.02, we sell     
     2.If return in the past 21 days < -0.02, we buy     
     3.We close the position every 10 days 


Momentum Strategy: 

    1.If return in the past 6 months > 0.05, we sell
    2.If return in the past 6 months < -0.05, we buy
    3.We close the position every month 
    
    
    


In [32]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

import pandas as pd
import datetime

In [47]:

class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

        
class DummyInd(bt.Indicator):
    lines = ('dummyline',)

    params = (('value', 5),)

    def __init__(self):
        self.lines.dummyline = bt.Max(0.0, self.params.value)

        
# Create a Stratey
class BackTradingStrategy(bt.Strategy):
    params = (
        ('period', 128),
        ("max_days_in_trade", 10)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
        
    def read_signals(self):
        signal_data = pd.read_csv("hmm_regime_signals.csv")
        signal_data["datetime"] = signal_data["datetime"].apply(lambda x: datetime.datetime.strptime(x,"%m/%d/%Y"))

        signal_data = signal_data.set_index('datetime')
        
        """
        start_date = '2013-02-11'
        end_date = '2018-02-09'
        signals = signal_data['state'].loc[start_date:end_date]               
        dt = self.datadate(0).strftime('%Y-%m-%d')
        print(signal_data.iloc[0])
        print(signal_data['state'].loc[dt])
        """
        
        return signal_data
        

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        # How to access individual etf
        self.xlb_close = self.datas[symbols.index("XLB")].close
        self.xle_close = self.datas[symbols.index("XLE")].close
        self.xlf_close = self.datas[symbols.index("XLF")].close
        self.xli_close = self.datas[symbols.index("XLI")].close
        self.xlk_close = self.datas[symbols.index("XLK")].close
        self.xlp_close = self.datas[symbols.index("XLP")].close
        self.xlu_close = self.datas[symbols.index("XLU")].close
        self.xlv_close = self.datas[symbols.index("XLV")].close
        self.xly_close = self.datas[symbols.index("XLY")].close
        
        self.datadate = self.datas[0].datetime.date
        self.days_in_trade = 0
        self.max_days_in_trade_mr = 10
        self.max_days_in_trade_mo = 21
        self.num_mr = 21
        self.num_mo = 21*6
        self.regime_state = 0
        self.mrr = 0.02
        self.mor = 0.05
        self.snum = 3
        self.signal_data = self.read_signals()
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)        
        self.symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
        
        
        
    def stock_return(self, n):
        """calculate return in the past n days """        
        r_xlb = (self.xlb_close[0] - self.xlb_close[-n])/self.xlb_close[-n]
        r_xle = (self.xle_close[0] - self.xle_close[-n])/self.xle_close[-n]
        r_xlf = (self.xlf_close[0] - self.xlf_close[-n])/self.xlf_close[-n]
        r_xli = (self.xli_close[0] - self.xli_close[-n])/self.xli_close[-n]
        r_xlk = (self.xlk_close[0] - self.xlk_close[-n])/self.xlk_close[-n]
        r_xlp = (self.xlp_close[0] - self.xlp_close[-n])/self.xlp_close[-n]
        r_xlu = (self.xlu_close[0] - self.xlu_close[-n])/self.xlu_close[-n]
        r_xlv = (self.xlv_close[0] - self.xlv_close[-n])/self.xlv_close[-n]
        r_xly = (self.xly_close[0] - self.xly_close[-n])/self.xly_close[-n]
        r_list = [r_xlb, r_xle, r_xlf, r_xli, r_xlk, r_xlp, r_xlu, r_xlv, r_xly]        
        
        df = pd.DataFrame(r_list, index = self.symbols, columns = ['return'])
        
        return df
    
    
    
    def sort_return(self, n):
        """sort return in the past n days in ascending order 
           return the bottom and top 3 return stocks """
        
        df = self.stock_return(n)
        df = df.sort_values(by=['return'])
                
        stock_list = [df.index[0:3].tolist(),df.index[6:].tolist()]
        
        return stock_list
    
    
    
    def check_position(self):
        flag = False
        names = []
        for symbol in self.symbols:
            s_size = self.getposition(data=self.datas[symbols.index(symbol)]).size
            if  s_size != 0:
                flag = True
                names.append(symbol)
        return flag, names
            
                
    

    def next(self):
        # Simply log the closing price of the series from the reference
        
        self.log('Close, %.2f' % self.xlb_close[0])        
        
        dt = self.datadate(0).strftime('%Y-%m-%d')
        
        signal = self.signal_data['state'].loc[dt]
        
        position, names = self.check_position()
        print(names)
        
        
        # Check if we are in the market
        if not position:
            # if we do not have a position
            
            if signal == 1:
                """mean reversion"""
                
                stock_list = self.sort_return(self.num_mr)
                # print('stocklist = ', stock_list)
                
                self.log('BUY CREATE, %.2f' % self.xlb_close[0])

                # Buy bottom snum stocks
                for i in range(self.snum):
                    self.order = self.buy(data = self.datas[symbols.index(stock_list[0][i])])
                    
                # Sell top snum stocks    
                for i in range(self.snum):
                    self.order = self.sell(data = self.datas[symbols.index(stock_list[1][i])])

                self.days_in_trade = 1
                
                self.regime_state = 1
                    
                    
         
            
            else:
                """momentum"""  
                
                stock_list = self.sort_return(self.num_mo)
                # print('stocklist = ', stock_list)
                
                self.log('SELL CREATE, %.2f' % self.xlb_close[0])

                # Sell bottom snum stocks
                for i in range(self.snum):
                    self.order = self.sell(data = self.datas[symbols.index(stock_list[0][i])])
                    
                # Buy top snum stocks    
                for i in range(self.snum):
                    self.order = self.buy(data = self.datas[symbols.index(stock_list[1][i])])
                    
                self.days_in_trade = 1
                
                self.regime_state = 0
                    
            
                
            

        else:
            # if we have a position
            
            self.days_in_trade += 1
            
            """mean reversion"""
            if (self.regime_state == 1):
            
                # close the position if regime shift or after holding it for max_mr days.
                if signal == 0 or self.days_in_trade == self.max_days_in_trade_mr:
                    num = len(names)
                    for i in range(num):
                        self.close(data=self.datas[symbols.index(names[i])])
                    
                    self.days_in_trade = 0                    
                    self.regime_state = 0 
                    
            
            """momentum"""
            if (self.regime_state == 0):
                
                # close the position if regime shift or after holding it for max_mo days.
                if signal == 1 or self.days_in_trade == self.max_days_in_trade_mo:
                    num = len(names)
                    for i in range(num):
                        self.close(data=self.datas[symbols.index(names[i])])

                    self.days_in_trade = 0                        
                    self.regime_state = 1
                 
       


In [48]:


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()   

    symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
    for symbol in symbols:
        datapath = os.path.join('../../../datas/%s.csv' % symbol)

        # Create a Data Feed
        data = bt.feeds.YahooFinanceCSVData(
            dataname=datapath,
            # Do not pass values before this date
            fromdate=datetime.datetime(2013, 2, 11),
            # Do not pass values before this date
            todate=datetime.datetime(2018, 2, 9),
            # Do not pass values after this date
            reverse=False)

        # Add the Data Feed to Cerebro
        cerebro.adddata(data, name=symbol)


    # Add a strategy
    cerebro.addstrategy(BackTradingStrategy) #, symbols= symbols)
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    

    # Write output
    cerebro.addwriter(bt.WriterFile, out='0402-ETFbacktesting.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2013-08-13, Close, 37.88
[]
       return
XLE  0.060663
XLB  0.066141
XLU  0.068152
XLK  0.093967
XLP  0.120522
XLI  0.132935
XLF  0.156035
XLV  0.186970
XLY  0.189670
stocklist =  [['XLE', 'XLB', 'XLU'], ['XLF', 'XLV', 'XLY']]
2013-08-13, SELL CREATE, 37.88
2013-08-14, Close, 37.80
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-15, Close, 37.53
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-16, Close, 37.23
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-19, Close, 36.94
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-20, Close, 37.15
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-21, Close, 36.87
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-22, Close, 37.31
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-23, Close, 37.61
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-26, Close, 37.61
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-27, Close, 36.95
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2013-08-28, Close, 36.88


['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-01, Close, 43.94
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-02, Close, 44.11
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-05, Close, 44.35
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-06, Close, 44.12
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-07, Close, 44.47
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-08, Close, 44.29
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-09, Close, 44.29
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-12, Close, 44.86
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-13, Close, 44.89
['XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLY']
2014-05-14, Close, 44.90
[]
       return
XLY  0.012594
XLP  0.056955
XLF  0.062138
XLK  0.090505
XLI  0.094144
XLE  0.102892
XLU  0.103931
XLV  0.106800
XLB  0.107275
stocklist =  [['XLY', 'XLP', 'XLF'], ['XLU', 'XLV', 'XLB']]
2014-05-14, SELL CREATE, 44.90
2014-05-15, Close, 44.27
['XLB', 'XLF', 'XLP', 'XLU', 'XLV', 'XLY']
2014-05-16, Close,

2015-01-16, Close, 45.08
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-20, Close, 45.16
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-21, Close, 45.60
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-22, Close, 46.17
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-23, Close, 45.46
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-26, Close, 45.66
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-27, Close, 45.34
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-28, Close, 44.53
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-29, Close, 45.11
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-01-30, Close, 44.93
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLV']
2015-02-02, Close, 45.34
[]
       return
XLE -0.178469
XLB -0.000661
XLK  0.049055
XLF  0.062260
XLI  0.080582
XLY  0.083775
XLP  0.133031
XLV  0.150908
XLU  0.193188
stocklist =  [['XLE', 'XLB', 'XLK'], ['XLP', 'XLV', 'XLU']]
2015-02-02, SELL CREATE, 45.34
2015-02-03, Close, 46.33
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 

       return
XLV -0.106407
XLB -0.096406
XLE -0.061956
XLF -0.057244
XLI -0.050948
XLY -0.035901
XLK -0.035558
XLP -0.013861
XLU -0.008058
stocklist =  [['XLV', 'XLB', 'XLE'], ['XLK', 'XLP', 'XLU']]
2015-09-28, BUY CREATE, 37.21
2015-09-29, Close, 37.35
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2015-09-30, Close, 38.16
[]
       return
XLE -0.199693
XLB -0.175097
XLI -0.090075
XLV -0.070650
XLF -0.050385
XLK -0.036999
XLP -0.020246
XLU -0.008408
XLY -0.003473
stocklist =  [['XLE', 'XLB', 'XLI'], ['XLP', 'XLU', 'XLY']]
2015-09-30, SELL CREATE, 38.16
2015-10-01, Close, 38.57
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLY']
2015-10-02, Close, 39.51
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLY']
2015-10-05, Close, 40.54
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLY']
2015-10-06, Close, 41.07
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLY']
2015-10-07, Close, 41.61
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLY']
2015-10-08, Close, 42.20
['XLB', 'XLE', 'XLI', 'XLP', 'XLU', 'XLY']
2015-10-09, Close, 42.18
['XLB', 'X

['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-10, Close, 44.77
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-11, Close, 44.62
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-12, Close, 44.81
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-13, Close, 44.38
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-16, Close, 45.14
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-17, Close, 44.91
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-18, Close, 44.22
['XLB', 'XLE', 'XLF', 'XLU', 'XLV', 'XLY']
2016-05-19, Close, 44.38
[]
       return
XLK -0.048469
XLY -0.033920
XLV -0.031570
XLI -0.029412
XLB -0.024186
XLF -0.021053
XLE -0.010443
XLP -0.005365
XLU  0.005311
stocklist =  [['XLK', 'XLY', 'XLV'], ['XLE', 'XLP', 'XLU']]
2016-05-19, BUY CREATE, 44.38
2016-05-20, Close, 44.70
['XLE', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
2016-05-23, Close, 45.22
[]
       return
XLF -0.048136
XLY -0.027337
XLK -0.018238
XLV -0.013440
XLE -0.008219
XLI  0.008184
XLB  0.036443
XLP  0.069513
X

['XLE', 'XLF', 'XLI', 'XLP', 'XLU', 'XLV']
2016-12-27, Close, 49.63
['XLE', 'XLF', 'XLI', 'XLP', 'XLU', 'XLV']
2016-12-28, Close, 49.13
['XLE', 'XLF', 'XLI', 'XLP', 'XLU', 'XLV']
2016-12-29, Close, 49.13
['XLE', 'XLF', 'XLI', 'XLP', 'XLU', 'XLV']
2016-12-30, Close, 48.81
['XLE', 'XLF', 'XLI', 'XLP', 'XLU', 'XLV']
2017-01-03, Close, 49.09
['XLE', 'XLF', 'XLI', 'XLP', 'XLU', 'XLV']
2017-01-04, Close, 49.81
['XLE', 'XLF', 'XLI', 'XLP', 'XLU', 'XLV']
2017-01-05, Close, 49.66
[]
       return
XLU -0.046733
XLP -0.042854
XLV -0.019972
XLY  0.056545
XLB  0.116708
XLI  0.131777
XLK  0.138346
XLE  0.147615
XLF  0.685902
stocklist =  [['XLU', 'XLP', 'XLV'], ['XLK', 'XLE', 'XLF']]
2017-01-05, SELL CREATE, 49.66
2017-01-06, Close, 49.71
['XLE', 'XLF', 'XLK', 'XLP', 'XLU', 'XLV']
2017-01-09, Close, 49.70
['XLE', 'XLF', 'XLK', 'XLP', 'XLU', 'XLV']
2017-01-10, Close, 49.73
['XLE', 'XLF', 'XLK', 'XLP', 'XLU', 'XLV']
2017-01-11, Close, 50.14
['XLE', 'XLF', 'XLK', 'XLP', 'XLU', 'XLV']
2017-01-12, Close,

2017-08-31, Close, 54.51
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2017-09-01, Close, 54.86
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2017-09-05, Close, 54.24
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2017-09-06, Close, 54.38
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2017-09-07, Close, 54.47
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2017-09-08, Close, 54.45
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2017-09-11, Close, 55.16
['XLB', 'XLE', 'XLK', 'XLP', 'XLU', 'XLV']
2017-09-12, Close, 55.63
[]
       return
XLE -0.042979
XLF  0.008614
XLP  0.024491
XLY  0.040493
XLI  0.071317
XLU  0.092049
XLB  0.093357
XLV  0.110336
XLK  0.120980
stocklist =  [['XLE', 'XLF', 'XLP'], ['XLB', 'XLV', 'XLK']]
2017-09-12, SELL CREATE, 55.63
2017-09-13, Close, 55.61
['XLB', 'XLE', 'XLF', 'XLK', 'XLP', 'XLV']
2017-09-14, Close, 55.79
['XLB', 'XLE', 'XLF', 'XLK', 'XLP', 'XLV']
2017-09-15, Close, 55.80
['XLB', 'XLE', 'XLF', 'XLK', 'XLP', 'XLV']
2017-09-18, Close, 56.10
['XLB', 'XLE', 'XLF', 'XLK', 'XLP', 

In [ ]:
# calculate pnl using excel data
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('backtesting.csv').fillna(value = 0)
data = data[:1260]

df = data[['datetime', 'pnlplus', 'pnlminus']].copy()
n = len(df)

df['cumulative_pnl'] = df['pnlplus']

cpnl = 0.
count_plus = 0
count_minus = 0

for i in range(n):
    plus = df['pnlplus'].iloc[i]
    minus = df['pnlminus'].iloc[i]
    cpnl = cpnl + plus + minus
    df['cumulative_pnl'].iloc[i] = cpnl
    
    if (plus > 0.):
        count_plus +=1
    if (minus < 0.):
        count_minus +=1

    

In [ ]:
plus_percent = 1.0*count_plus/(count_plus*1.0+count_minus*1.0) 
print('percentage of trading that make positive profits = ', plus_percent)

In [ ]:
# convert string to datetime
df["datetime"] = df["datetime"].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f"))

# plot PNL figures 
plt.figure(figsize=(16, 4))
plt.plot(df['datetime'], df['cumulative_pnl'] ,'.') 
plt.xlabel("Time")
plt.ylabel("cumulative pnl")
plt.title("Daily PNL")
plt.show()